In [156]:
#Chiefs Data Scrape using http://pro-football-reference.com


import pandas as pd
from bs4 import BeautifulSoup as bs
import sys 
import os
from urllib.request import urlopen
import sklearn
import numpy as np
import re

import time


#change max_columns in pandas

pd.set_option('max_columns', 50)

#start scipt clock
time1 = time.time()

print(sys.version)

3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]


In [157]:
#determine how many years of data you would like to scrape

years = list(range(2005,2019))

years[-1]

2018

In [158]:
print(sys.version)

3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]


In [0]:
#single year pull
url = 'https://www.pro-football-reference.com/teams/kan/2017.htm'

In [0]:
page = urlopen(url)
soup = bs(page)
table = soup.find_all("table")
table = table[1]

In [0]:
#pull for all years
for year in years:
    if year == years[0]:
        predf = dict()
        week = 0
        urls = 'https://www.pro-football-reference.com/teams/kan/'
        end = '.htm'
        url = urls+str(year)+end
        page = urlopen(url).read()
        soup = bs(page)
        table = soup.find_all("table")
        table = table[1]
        rows = table.find_all('tr')
        for row in rows:
            a = row.find_all('td')
            for rowrow in a:
                b = rowrow.text.strip().encode()
                c = b.decode('utf-8')
                ff = rowrow['data-stat']
                if ff in predf:
                    predf[ff].append(c)
                else:
                    predf[ff] = [c]
        df1 = pd.DataFrame.from_dict(predf)
        aa = []
        for index,row in df1.iterrows():
            aa.append(index + 1)
        df1['week'] = aa
        df1['season'] = year
    else:
        predf = dict()
        urls = 'https://www.pro-football-reference.com/teams/kan/'
        end = '.htm'
        url = urls+str(year)+end
        page = urlopen(url).read()
        soup = bs(page)
        table = soup.find_all("table")
        table = table[1]
        rows = table.find_all('tr')
        for row in rows:
            a = row.find_all('td')
            for rowrow in a:
                b = rowrow.text.strip().encode()
                c = b.decode('utf-8')
                ff = rowrow['data-stat']
                if ff in predf:
                    predf[ff].append(c)
                else:
                    predf[ff] = [c]
        df2 = pd.DataFrame.from_dict(predf)
        aa = []
        for index,row in df2.iterrows():
            aa.append(index + 1)
        df2['week'] = aa
        df2['season'] = year
        df1 = pd.concat([df1,df2],axis = 0,ignore_index=True)
            

In [0]:
#rearrange columns so season and week are first
df1 = pd.concat([df1[['season','week']],df1.iloc[:,:-2]],axis = 1)

In [0]:
df1.iloc[176:178,:]

In [0]:
#add feature for playoff games
df1['playoff_game'] = 'no'
aa = []
for index, row in df1.iterrows():
    if row['game_location'] == '@':
        aa.append('away')
    else:
        aa.append('home')
df1['game_location'] = aa

In [0]:
for index, row in df1.iterrows():
    if row['week'] > 17:
        df1.loc[index,'week'] = df1.loc[index,'week'] - 1
        df1.loc[index,'playoff_game'] = 'yes'
        

In [0]:
df1 = df1[df1['game_date'] != 'Playoffs']
del[df1['boxscore_word']]

In [0]:
print('total script time', round((time.time() - time1) / 60,2),'minutes')

In [0]:
#get attendance html

url = 'https://www.pro-football-reference.com/years/2006/attendance.htm'
page = urlopen(url).read()

soup = bs(page)

In [0]:
table = soup.find("table", {'id' : 'attendance'})

In [0]:
predf = dict()
rows = table.find_all('tr')


for row in rows:
  b = row.find_all('th')[0].text.strip()
  if b == 'Tm':
    pass
  elif 'team' not in predf:
    print(b)
    predf['team'] = [b]
  else:
    predf['team'].append(b)

for row in rows:
  a = row.find_all('td')
  for rowrow in a:
      b = rowrow.text.strip().encode()
      c = b.decode('utf-8')
      ff = rowrow['data-stat']
      if ff in predf:
          predf[ff].append(c)
      else:
          predf[ff] = [c]

In [0]:
a = pd.DataFrame(predf)

aa = ['team']

for n in list(range(1,18)):
  aa.append('week'+str(n))
  
b = a[a['team'] == 'Kansas City Chiefs'][aa].transpose().reset_index()
 
b.columns = ['week','attendance']


b = b.iloc[1:,:].reset_index()

del[b['index']]

b.head()

In [0]:
#make dataset of all years for attendance

for year in years:
  if year == years[0]:
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/attendance.htm'
    page = urlopen(url).read()
    soup = bs(page)
    table = soup.find("table", {'id' : 'attendance'})
    predf = dict()
    rows = table.find_all('tr')

    for row in rows:
      b = row.find_all('th')[0].text.strip()
      if b == 'Tm':
        pass
      elif 'team' not in predf:
        #print(b)
        predf['team'] = [b]
      else:
        predf['team'].append(b)

    for row in rows:
      a = row.find_all('td')
      for rowrow in a:
          b = rowrow.text.strip().encode()
          c = b.decode('utf-8')
          ff = rowrow['data-stat']
          if ff in predf:
              predf[ff].append(c)
          else:
              predf[ff] = [c]
    a = pd.DataFrame(predf)

    aa = ['team']

    for n in list(range(1,18)):
      aa.append('week'+str(n))

    attend = a[a['team'] == 'Kansas City Chiefs'][aa].transpose().reset_index()

    attend.columns = ['week','attendance']


    attend = attend.iloc[1:,:].reset_index()

    del[attend['index']]
    attend['year'] = year
  else:
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/attendance.htm'
    page = urlopen(url).read()
    soup = bs(page)
    table = soup.find("table", {'id' : 'attendance'})
    predf = dict()
    rows = table.find_all('tr')

    for row in rows:
      b = row.find_all('th')[0].text.strip()
      if b == 'Tm':
        pass
      elif 'team' not in predf:
        #print(b)
        predf['team'] = [b]
      else:
        predf['team'].append(b)

    for row in rows:
      a = row.find_all('td')
      for rowrow in a:
          b = rowrow.text.strip().encode()
          c = b.decode('utf-8')
          ff = rowrow['data-stat']
          if ff in predf:
              predf[ff].append(c)
          else:
              predf[ff] = [c]
    a = pd.DataFrame(predf)

    aa = ['team']

    for n in list(range(1,18)):
      aa.append('week'+str(n))

    attend1 = a[a['team'] == 'Kansas City Chiefs'][aa].transpose().reset_index()

    attend1.columns = ['week','attendance']


    attend1 = attend1.iloc[1:,:].reset_index()

    del[attend1['index']]
    attend1['year'] = year
    attend = pd.concat([attend,attend1], axis = 0)
    




In [0]:
attend.head()

In [0]:
#make unique key out of year and week

aa = []

for index,row in attend.iterrows():
  year = str(row['year'])
  week = re.sub('[a-z]', '', row['week'])
  year_week = year + week
  aa.append(year_week)
  
attend['year_week'] = aa

In [0]:
attend.head()

In [0]:
df1.head()

In [0]:
#create year_week for main dataframe

aa = [] 

for index,row in df1.iterrows():
  year = str(row['season'])
  week = str(row['week'])
  aa.append(year + week)
  
df1['year_week'] = aa

In [0]:
df = df1.merge(attend[['attendance','year_week']], on = 'year_week')

#delete matching key
del[df['year_week']]

In [0]:
df.tail()